In [1]:
import pandas as pd
import numpy as np
import cplex
from cplex.exceptions import CplexError

In [2]:
# Load data from the Excel file
file_path = 'OriginalDataset.xlsx'
data = pd.read_excel(file_path, sheet_name=None)

In [3]:
sets_df = data.get('Sets', pd.DataFrame())
warehouse = sets_df[sets_df['Type'] == 'Warehouse']
vehicle = sets_df[sets_df['Type'] == 'Vehicle']
customer = sets_df[sets_df['Type'] == 'Customer']
pickup = sets_df[sets_df['Type'] == 'Pickup']
delivery = sets_df[sets_df['Type'] == 'Delivery']

In [4]:
delivery

,Type,ID
25,Delivery,C1
26,Delivery,C2
27,Delivery,C3
28,Delivery,C4
29,Delivery,C5
30,Delivery,C6
31,Delivery,C7
32,Delivery,C8
33,Delivery,C9
34,Delivery,C10


In [5]:
transportcost = data.get('TransportationCost', pd.DataFrame())
carbon_emissions = data.get('CarbonEmissions', pd.DataFrame())
pickupdemands = data.get('PickupDemand', pd.DataFrame())
deliverydemands = data.get('DeliveryDemand', pd.DataFrame())
vehicle_capacity = data.get('VehicleCapacity', pd.DataFrame())

n_customers = 16
Ccustomer = customer.head(n_customers)
Cpickup = pickup[pickup['ID'].isin(Ccustomer['ID'])]
Cdelivery = delivery[delivery['ID'].isin(Ccustomer['ID'])]

Ce = carbon_emissions[carbon_emissions['Unnamed: 0'].isin(Ccustomer['ID'])]

# Sets and indices
W = set(warehouse['ID'])
K = set(vehicle['ID'])
C = set(Ccustomer['ID'])
Cd = set(Cdelivery['ID'])
Cp = set(Cpickup['ID'])

In [6]:
Ce

,Unnamed: 0,W1,C1,C2,C3,C4,C5,C6,C7,C8,...,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25
1,C1,0.369101,0.364495,0.367597,0.367174,0.367362,0.366328,0.366469,0.368819,0.368443,...,0.368255,0.367926,0.366563,0.367362,0.365388,0.368302,0.366939,0.366939,0.367409,0.366704
2,C2,0.369101,0.367597,0.364495,0.366516,0.368067,0.369101,0.368208,0.366751,0.368255,...,0.368020,0.368490,0.367691,0.366657,0.368819,0.366939,0.368302,0.368020,0.368819,0.367033
3,C3,0.366892,0.367174,0.366516,0.364495,0.369101,0.367315,0.365811,0.365388,0.367409,...,0.367926,0.365952,0.366281,0.368349,0.368208,0.367926,0.366892,0.366281,0.367738,0.366657
4,C4,0.366939,0.367362,0.368067,0.369101,0.364495,0.368396,0.367456,0.368819,0.367879,...,0.366140,0.366845,0.365905,0.369054,0.365388,0.368020,0.367409,0.365999,0.365012,0.367644
5,C5,0.368490,0.366328,0.369101,0.367315,0.368396,0.364495,0.366328,0.368208,0.369054,...,0.367691,0.367268,0.367503,0.368537,0.367597,0.368725,0.366046,0.368537,0.368631,0.366375
6,C6,0.365764,0.366469,0.368208,0.365811,0.367456,0.366328,0.364495,0.365482,0.367832,...,0.369101,0.366845,0.364965,0.367362,0.369101,0.368114,0.368772,0.368396,0.368443,0.368302
7,C7,0.368819,0.368819,0.366751,0.365388,0.368819,0.368208,0.365482,0.364495,0.365153,...,0.367738,0.367926,0.368678,0.365717,0.368208,0.366845,0.365764,0.366892,0.368443,0.365106
8,C8,0.368114,0.368443,0.368255,0.367409,0.367879,0.369054,0.367832,0.365153,0.364495,...,0.367409,0.367127,0.368302,0.367926,0.365999,0.367409,0.368631,0.368725,0.367644,0.367033
9,C9,0.369101,0.365670,0.369007,0.368067,0.366187,0.367597,0.366939,0.368725,0.369101,...,0.368537,0.367503,0.367456,0.368960,0.368866,0.368678,0.366422,0.367315,0.366046,0.366986
10,C10,0.365905,0.368725,0.366187,0.369101,0.367973,0.368020,0.368678,0.367362,0.368631,...,0.369007,0.365999,0.368772,0.366328,0.368960,0.365905,0.366704,0.367409,0.367503,0.367268


In [7]:
# Parameters
locations = list(C.union(W))
cij = {}
eij = {}

for k in K:
    for i in locations:
        for j in locations:
            cij[(i, j, k)] = float(transportcost.loc[transportcost.iloc[:, 0] == i, str(j)].values[0])
            eij[(i, j, k)] = float(carbon_emissions.loc[carbon_emissions.iloc[:, 0] == i, str(j)].values[0])

In [8]:
cij

{('C5', 'C5', 'V1'): 0.0,
 ('C5', 'C3', 'V1'): 45.0,
 ('C5', 'C9', 'V1'): 49.5,
 ('C5', 'C12', 'V1'): 68.25,
 ('C5', 'C15', 'V1'): 68.25,
 ('C5', 'C8', 'V1'): 72.75,
 ('C5', 'C13', 'V1'): 18.0,
 ('C5', 'C16', 'V1'): 51.0,
 ('C5', 'C11', 'V1'): 66.0,
 ('C5', 'C14', 'V1'): 60.0,
 ('C5', 'C1', 'V1'): 29.25,
 ('C5', 'C7', 'V1'): 59.25,
 ('C5', 'C10', 'V1'): 56.25,
 ('C5', 'C4', 'V1'): 62.25,
 ('C5', 'C2', 'V1'): 73.5,
 ('C5', 'C6', 'V1'): 29.25,
 ('C5', 'W1', 'V1'): 63.75,
 ('C3', 'C5', 'V1'): 45.0,
 ('C3', 'C3', 'V1'): 0.0,
 ('C3', 'C9', 'V1'): 57.0,
 ('C3', 'C12', 'V1'): 11.25,
 ('C3', 'C15', 'V1'): 67.5,
 ('C3', 'C8', 'V1'): 46.5,
 ('C3', 'C13', 'V1'): 59.25,
 ('C3', 'C16', 'V1'): 54.75,
 ('C3', 'C11', 'V1'): 60.75,
 ('C3', 'C14', 'V1'): 66.75,
 ('C3', 'C1', 'V1'): 42.75,
 ('C3', 'C7', 'V1'): 14.25,
 ('C3', 'C10', 'V1'): 73.5,
 ('C3', 'C4', 'V1'): 73.5,
 ('C3', 'C2', 'V1'): 32.25,
 ('C3', 'C6', 'V1'): 21.0,
 ('C3', 'W1', 'V1'): 38.25,
 ('C9', 'C5', 'V1'): 49.5,
 ('C9', 'C3', 'V1'): 57.0

In [9]:
di = deliverydemands.set_index('CustomerID').to_dict()['Demand']
pi = pickupdemands.set_index('CustomerID').to_dict()['Demand']
Qk = vehicle_capacity.set_index('VehicleID').to_dict()['Capacity']

In [10]:
pi

{'C3': 1.531184,
 'C6': 6.93072,
 'C11': 0.343479,
 'C13': 3.129383,
 'C19': 0.503502}

In [12]:
alpha = 0.0733
Emax = 1000

In [13]:
# Initialize the model
model = cplex.Cplex()
model.set_problem_type(cplex.Cplex.problem_type.MILP)

# Decision variables
x = {}
u = {}

for k in K:
    for i in locations:
        for j in locations:
            var_name = f"x_{i}_{j}_{k}"
            x[(i, j, k)] = var_name
            model.variables.add(names=[var_name], types=[model.variables.type.binary])
        u[(i, k)] = f"u_{i}_{k}"
        model.variables.add(names=[u[(i, k)]], types=[model.variables.type.continuous])

# Adjust the objective function to minimize both cost and emissions
objective_vars = []
objective_coeffs = []

for k in K:
    for i in C:
        for j in C:
            if i != j:
                objective_vars.append(x[(i, j, k)])
                objective_coeffs.append(cij[(i, j, k)] + eij[(i, j, k)])  # Combine cost and emissions

# Minimize the objective
model.objective.set_linear(zip(objective_vars, objective_coeffs))
model.objective.set_sense(model.objective.sense.minimize)

In [14]:
locations

['C5',
 'C3',
 'C9',
 'C12',
 'C15',
 'C8',
 'C13',
 'C16',
 'C11',
 'C14',
 'C1',
 'C7',
 'C10',
 'C4',
 'C2',
 'C6',
 'W1']

In [15]:
# Each customer is visited exactly once by one vehicle
for i in C: 
    lhs_vars = [x[(j, i, k)] for j in locations for k in K if j != i]  # Variables indicating visits to customer i
    lhs_coeffs = [1] * len(lhs_vars)  # Coefficients for the linear expression (all 1s)
    print(f"Customer {i} visit constraint: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs=1")
    name = "Customer visited" 
    model.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
        senses=["E"],  
        rhs=[1],names=[name]  
    )

# Vehicles must start and end at a warehouse
for k in K:
    start_constraint_expr = cplex.SparsePair(ind=[], val=[])
    for w in W:
        for j in C:
            start_constraint_expr.ind.append(x[(w, j, k)])
            start_constraint_expr.val.append(1)
    print(f"Vehicle {k} start constraint: vars={start_constraint_expr.ind}, coeffs={start_constraint_expr.val}, rhs=1")
    name = "Start at W"
    model.linear_constraints.add(
        lin_expr=[start_constraint_expr],
        senses=["E"], rhs=[1],names=[name]
    )
        
for k in K:
    end_constraint_expr = cplex.SparsePair(ind=[], val=[])
    for w in W:
        for i in C:            
            end_constraint_expr.ind.append(x[(i, w, k)])
            end_constraint_expr.val.append(1)
    print(f"Vehicle {k} end constraint: vars={end_constraint_expr.ind}, coeffs={end_constraint_expr.val}, rhs=1")
    name = "End at W"
    model.linear_constraints.add(
        lin_expr=[end_constraint_expr],
        senses=["E"], rhs=[1], names=[name]
    )

Customer C11 visit constraint: vars=['x_C5_C11_V1', 'x_C5_C11_V2', 'x_C3_C11_V1', 'x_C3_C11_V2', 'x_C9_C11_V1', 'x_C9_C11_V2', 'x_C12_C11_V1', 'x_C12_C11_V2', 'x_C15_C11_V1', 'x_C15_C11_V2', 'x_C8_C11_V1', 'x_C8_C11_V2', 'x_C13_C11_V1', 'x_C13_C11_V2', 'x_C16_C11_V1', 'x_C16_C11_V2', 'x_C14_C11_V1', 'x_C14_C11_V2', 'x_C1_C11_V1', 'x_C1_C11_V2', 'x_C7_C11_V1', 'x_C7_C11_V2', 'x_C10_C11_V1', 'x_C10_C11_V2', 'x_C4_C11_V1', 'x_C4_C11_V2', 'x_C2_C11_V1', 'x_C2_C11_V2', 'x_C6_C11_V1', 'x_C6_C11_V2', 'x_W1_C11_V1', 'x_W1_C11_V2'], coeffs=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], rhs=1
Customer C14 visit constraint: vars=['x_C5_C14_V1', 'x_C5_C14_V2', 'x_C3_C14_V1', 'x_C3_C14_V2', 'x_C9_C14_V1', 'x_C9_C14_V2', 'x_C12_C14_V1', 'x_C12_C14_V2', 'x_C15_C14_V1', 'x_C15_C14_V2', 'x_C8_C14_V1', 'x_C8_C14_V2', 'x_C13_C14_V1', 'x_C13_C14_V2', 'x_C16_C14_V1', 'x_C16_C14_V2', 'x_C11_C14_V1', 'x_C11_C14_V2', 'x_C1_C14_V1', 'x_C1_C14_V2', 'x_C7_C14_V1

In [16]:
# Flow conservation
for k in K:
    for i in C:
        incoming = [x[(j, i, k)] for j in locations if j != i]
        outgoing = [x[(i, j, k)] for j in locations if j != i]
        print(f"Flow conservation for vehicle {k} at location {i}: incoming={incoming}, outgoing={outgoing}")
        name="Flow Conservation"   
        model.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=incoming + outgoing, val=[1] * len(incoming) + [-1] * len(outgoing))],
                senses=["E"],
                rhs=[0],names=[name]
            )
        
# MTZ Subtour elimination
for k in K:
    for i in C:
        for j in C:
            lhs_vars=[]
            lhs_coeffs=[]
            if i != j:
                if j not in pi:
                    pi[j] = 0
                if j not in di:
                    di[j] = 0
                lhs_vars.append("u_" +str(i)+"_"+str(k))
                lhs_coeffs.append(1)
                lhs_vars.append("u_" +str(j)+"_"+str(k))
                lhs_coeffs.append(-1)
                lhs_vars.append("x_" +str(i)+"_"+str(j)+"_"+str(k))
                lhs_coeffs.append(Qk[k])
                rhs = Qk[k] - di[j] + pi[j]
                print(f"MTZ constraint for vehicle {k} from {i} to {j}: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs={rhs}")
                name = "Subtour1"
                model.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
                    senses=["L"],
                    rhs=[rhs], names=[name]
                )
            
# Capacity constraint
for k in K:
    for i in C:
        expr_vars=[]
        expr_coeffs=[]
        if i in pi:
            pickup = pi[i]
        else:
            pickup = 0
        if i in di:
            delivery = di[i]
        else:
            delivery = 0

        expr_vars.append("u_" +str(i)+"_"+str(k))
        expr_coeffs.append(1)
        print(f"Capacity constraint for vehicle {k} at customer {i}: vars={expr_vars}, coeffs={expr_coeffs}, rhs={Qk[k]}")
        name = "capacity"
        model.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=expr_vars, val=expr_coeffs)],
            senses=["L"],
            rhs=[Qk[k]], names=[name]
        )
    
model.parameters.mip.tolerances.mipgap.set(0.01)  # For example, set MIP gap tolerance

Flow conservation for vehicle V1 at location C11: incoming=['x_C5_C11_V1', 'x_C3_C11_V1', 'x_C9_C11_V1', 'x_C12_C11_V1', 'x_C15_C11_V1', 'x_C8_C11_V1', 'x_C13_C11_V1', 'x_C16_C11_V1', 'x_C14_C11_V1', 'x_C1_C11_V1', 'x_C7_C11_V1', 'x_C10_C11_V1', 'x_C4_C11_V1', 'x_C2_C11_V1', 'x_C6_C11_V1', 'x_W1_C11_V1'], outgoing=['x_C11_C5_V1', 'x_C11_C3_V1', 'x_C11_C9_V1', 'x_C11_C12_V1', 'x_C11_C15_V1', 'x_C11_C8_V1', 'x_C11_C13_V1', 'x_C11_C16_V1', 'x_C11_C14_V1', 'x_C11_C1_V1', 'x_C11_C7_V1', 'x_C11_C10_V1', 'x_C11_C4_V1', 'x_C11_C2_V1', 'x_C11_C6_V1', 'x_C11_W1_V1']
Flow conservation for vehicle V1 at location C14: incoming=['x_C5_C14_V1', 'x_C3_C14_V1', 'x_C9_C14_V1', 'x_C12_C14_V1', 'x_C15_C14_V1', 'x_C8_C14_V1', 'x_C13_C14_V1', 'x_C16_C14_V1', 'x_C11_C14_V1', 'x_C1_C14_V1', 'x_C7_C14_V1', 'x_C10_C14_V1', 'x_C4_C14_V1', 'x_C2_C14_V1', 'x_C6_C14_V1', 'x_W1_C14_V1'], outgoing=['x_C14_C5_V1', 'x_C14_C3_V1', 'x_C14_C9_V1', 'x_C14_C12_V1', 'x_C14_C15_V1', 'x_C14_C8_V1', 'x_C14_C13_V1', 'x_C14_C16_V

In [17]:
#Emissions constraint 
lhs_vars = []
lhs_coeffs = []

# eij * xijk
for k in K:
    for i in locations:  # Use locations to include both warehouses and customers
        for j in locations:
            if i != j:
                lhs_vars.append(x[(i, j, k)])
                lhs_coeffs.append(eij[(i, j, k)])
                print(f"Adding x[{i}, {j}, {k}] with coefficient {eij[(i, j, k)]}")

# alpha * dij * uik
for k in K:
    for i in locations:
        if i in di:
            delivery = di[i]
        else:
            delivery = 0
        lhs_vars.append(u[(i, k)])
        lhs_coeffs.append(alpha * delivery)
        print(f"Adding u[{i}, {k}] with coefficient {alpha * delivery}")

print(f"Total emission constraint: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs={Emax}")
name = "Emissions"
model.linear_constraints.add(
    lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
    senses=["L"],
    rhs=[Emax], names=[name]
)

Adding x[C5, C3, V1] with coefficient 0.367315084
Adding x[C5, C9, V1] with coefficient 0.3675970924
Adding x[C5, C12, V1] with coefficient 0.3687721274
Adding x[C5, C15, V1] with coefficient 0.3687721274
Adding x[C5, C8, V1] with coefficient 0.3690541358
Adding x[C5, C13, V1] with coefficient 0.3656230336
Adding x[C5, C16, V1] with coefficient 0.3676910952
Adding x[C5, C11, V1] with coefficient 0.3686311232
Adding x[C5, C14, V1] with coefficient 0.368255112
Adding x[C5, C1, V1] with coefficient 0.3663280546
Adding x[C5, C7, V1] with coefficient 0.3682081106
Adding x[C5, C10, V1] with coefficient 0.368020105
Adding x[C5, C4, V1] with coefficient 0.3683961162
Adding x[C5, C2, V1] with coefficient 0.3691011372
Adding x[C5, C6, V1] with coefficient 0.3663280546
Adding x[C5, W1, V1] with coefficient 0.368490119
Adding x[C3, C5, V1] with coefficient 0.367315084
Adding x[C3, C9, V1] with coefficient 0.3680671064
Adding x[C3, C12, V1] with coefficient 0.365200021
Adding x[C3, C15, V1] with co

range(564, 565)

In [18]:
### Solve the model
model.parameters.mip.tolerances.mipgap.set(0.01)  # Set MIP gap tolerance
model.solve()

model.write("output.lp")
# Output the solution
solution = model.solution
if solution.is_primal_feasible():
    total_cost_all_vehicles = 0  # Initialize total cost for all vehicles
    total_emissions_all_vehicles = 0  # Initialize total emissions for all vehicles
    for k in K:
        route = []
        total_cost = 0  # Initialize total cost for the vehicle
        total_emissions = 0  # Initialize total emissions for the vehicle
        for i in locations:
            print(f"u[{i}, {k}] = {solution.get_values(u[(i, k)])}")
            print(f"--------")
            for j in locations:
                if i != j and solution.get_values(x[(i, j, k)]) > 0.5:
                    route.append((i, j))
                    cost = cij[(i, j, k)]
                    emissions = eij[(i, j, k)]
                    total_cost += cost
                    total_emissions += emissions
                    print(f"Route taken: c[{i}, {j}, {k}] = {cost}, e[{i}, {j}, {k}] = {emissions}")
                    print(f"--------")
        if route:
            print(f"Vehicle {k} route: {route}")
            print("--------")
            print(f"Total cost for vehicle {k}: {total_cost}")
            print(f"Total emissions for vehicle {k}: {total_emissions}")
        total_cost_all_vehicles += total_cost
        total_emissions_all_vehicles += total_emissions
    print(f"--------")
    print(f"Total cost for all vehicles: {total_cost_all_vehicles}")
    print(f"Total emissions for all vehicles: {total_emissions_all_vehicles}")
else:
    print("No feasible solution found.")

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
Tried aggregator 1 time.
MIP Presolve eliminated 32 rows and 36 columns.
MIP Presolve modified 604 coefficients.
Reduced MIP has 533 rows, 576 columns, and 3542 nonzeros.
Reduced MIP has 544 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (2.97 ticks)
Probing time = 0.00 sec. (2.54 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 533 rows, 576 columns, and 3542 nonzeros.
Reduced MIP has 544 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (2.11 ticks)
Probing time = 0.00 sec. (2.53 ticks)
Clique table members: 2134.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (1.07 ticks)

        Nodes                                     

u[C5, V1] = 66.866242
--------
Route taken: c[C5, C6, V1] = 29.25, e[C5, C6, V1] = 0.3663280546
--------
u[C3, V1] = 0.0
--------
u[C9, V1] = 27.03242499999999
--------
Route taken: c[C9, C11, V1] = 9.75, e[C9, C11, V1] = 0.3651060182
--------
u[C12, V1] = 0.0
--------
u[C15, V1] = 0.0
--------
Route taken: c[C15, C14, V1] = 36.75, e[C15, C14, V1] = 0.3667980686
--------
u[C8, V1] = 117.429308
--------
Route taken: c[C8, W1, V1] = 57.75, e[C8, W1, V1] = 0.3681141078
--------
u[C13, V1] = 39.985330000000005
--------
Route taken: c[C13, C1, V1] = 14.25, e[C13, C1, V1] = 0.3653880266
--------
u[C16, V1] = 0.0
--------
u[C11, V1] = 39.99452199999999
--------
Route taken: c[C11, C13, V1] = 15.75, e[C11, C13, V1] = 0.3654820294
--------
u[C14, V1] = 24.975292999999994
--------
Route taken: c[C14, C9, V1] = 15.0, e[C14, C9, V1] = 0.365435028
--------
u[C1, V1] = 55.37530799999999
--------
Route taken: c[C1, C5, V1] = 29.25, e[C1, C5, V1] = 0.3663280546
--------
u[C7, V1] = 89.90214800000001
-